## Timetabling Program for West Point Grey Academy

In [1]:
#import sys
#!{sys.executable} -m pip install ortools 

# import modules
import time
import numpy as np
import pandas as pd
from ortools.linear_solver import pywraplp

In [2]:
# Import Course Data
CourseInput = pd.ExcelFile("WPGA Course Data.xlsx") 
CourseMatrix = pd.read_excel(CourseInput, 'Data') 
CourseInfo = CourseMatrix.values.tolist()
CourseMatrix[0:5]

,Course,CourseID,Section,Teacher,Fixed Block,Room,Code,ConflictCode
0,AP Biology,1,1,Butler,NaN,Science Lab,APBIO12,1.0
1,AP Comparative Government and Politics,2,1,Lee,NaN,NaN,APCGP12,NaN
2,AP Chemistry,3,1,Bohnen,NaN,Science Lab,APCHE12,1.0
3,AP Chinese Language and Culture,4,1,Lei,NaN,021W,APCLC12,NaN
4,AP French Language and Culture,5,1,NaN,NaN,NaN,APFRL12,NaN


In [3]:
# Optimize assignment of courses to blocks

solver = pywraplp.Solver('West Point Grey Academy', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

start_time = time.time()

Sections = [1,2,3,4,5]
Blocks = [1,2,3,4,5,6,7,8,9]

# Find the number of sections offered in each course

Courses=[]
CourseSections = [ [] for m in range(len(CourseInfo))]

for m in range(len(CourseInfo)):
    CourseID = CourseInfo[m][1]
    CourseSection = CourseInfo[m][2]
    CourseSections[CourseID].append(CourseSection)        

for m in range(len(CourseInfo)):
    if len(CourseSections[m])>0:
        Courses.append(m)


# define boolean variables
x = {}
for s in Sections:
    for j in Courses:
        for k in Blocks:
            x[s,j,k] = solver.IntVar(0,1, 'x[%d,%d,%d]' % (s,j,k))

            
# CONSTRAINT 1: Ensure the right number of sections for each course

for j in Courses:
    for s in Sections:
        if s in CourseSections[j]:
            solver.Add(sum(x[s,j,k] for k in Blocks) == 1)
        else:
            solver.Add(sum(x[s,j,k] for k in Blocks) == 0)
        

# CONSTRAINT 2: Two sections of the same course can't be offered in the same block
for j in Courses:
    for k in Blocks:
        solver.Add(x[1,j,k] + x[2,j,k] + x[3,j,k] + x[4,j,k] + x[5,j,k] <= 1)

        
# CONSTRAINT 3: At most eleven classes per block
for k in Blocks:
    solver.Add(sum(x[1,j,k] + x[2,j,k] + x[3,j,k] + x[4,j,k] + x[5,j,k] for j in Courses) <= 11)
    

# CONSTRAINT 4: No teacher can teach two classes in the same block
TeacherList = set([CourseInfo[z][3] for z in range(len(CourseInfo))])
for Teacher in TeacherList:
    CourseList=[]
    for z in range(len(CourseInfo)):
        if CourseInfo[z][3]==Teacher:
            CourseList.append([CourseInfo[z][2],CourseInfo[z][1]])
    if len(CourseList)>1:
        for k in Blocks:
            solver.Add(sum(x[CourseList[j][0],CourseList[j][1],k] for j in range(len(CourseList))) <= 1)
        
        
# CONSTRAINT 5: No room can be used twice in the same block
RoomList = set([CourseInfo[z][5] for z in range(len(CourseInfo))])
for Room in RoomList:
    CourseList=[]
    for z in range(len(CourseInfo)):
        if CourseInfo[z][5]==Room:
            CourseList.append([CourseInfo[z][2],CourseInfo[z][1]])
    if len(CourseList)>1:
        for k in Blocks:
            solver.Add(sum(x[CourseList[j][0],CourseList[j][1],k] for j in range(len(CourseList))) <= 1)
        
        
# CONSTRAINT 6: Ensure certain pre-defined courses are in fixed blocks
for z in range(len(CourseInfo)):
    if CourseInfo[z][4] in Blocks:
        s = CourseInfo[z][2]
        j = CourseInfo[z][1]
        k = CourseInfo[z][4]
        solver.Add(x[s,j,k] == 1)
        

# CONSTRAINT 7: No pairs of Conflicting Courses can be scheduled together

ConflictList = set([CourseInfo[z][7] for z in range(len(CourseInfo))])
for Conflict in ConflictList:
    CourseList=[]
    for z in range(len(CourseInfo)):
        if CourseInfo[z][7]==Conflict:
            CourseList.append([CourseInfo[z][2],CourseInfo[z][1]])
    if len(CourseList)>1:
        for k in Blocks:
            solver.Add(sum(x[CourseList[j][0],CourseList[j][1],k] for j in range(len(CourseList))) <= 1)
        
        
solver.Maximize(solver.Sum(x[s,j,k] for s in Sections for j in Courses for k in Blocks))
sol = solver.Solve()
print("")
print('Optimization Complete with Total Happiness Score of', round(solver.Objective().Value()))

# compute runtime
solving_time = time.time() - start_time

print('The code ran in', round(solving_time,1), 'seconds')



Optimization Complete with Total Happiness Score of 95
The code ran in 0.3 seconds


In [4]:
# Print the courses scheduled in each block 

OutputPair = [[ [],[],[],[],[],[] ] for j in range(len(Courses)+1)]
for i in range(len(CourseInfo)):
    CourseName = CourseInfo[i][0]
    CourseID = CourseInfo[i][1]
    SectionID = CourseInfo[i][2]
    TeacherName = CourseInfo[i][3]
    
    if not TeacherName in list(TeacherList)[1:18]:
        TeacherName = "Unassigned Teacher"
     
    OutputPair[CourseID][SectionID] = [CourseName, TeacherName]
    
    
for k in Blocks:
    print()
    print("Block", k, "courses are")

    for j in Courses:
        for s in Sections:
            if x[s,j,k].solution_value()==1:
                print("Course", j, "Section", s, OutputPair[j][s])
        


Block 1 courses are
Course 7 Section 2 ['AP Psychology', 'Lee']
Course 10 Section 3 ['Career-Life Education', 'Garinger, D']
Course 15 Section 1 ['Destination Imagination 9', 'Bendl']
Course 21 Section 1 ['English 8', 'Unassigned Teacher']
Course 22 Section 4 ['English 9', 'Unassigned Teacher']
Course 26 Section 1 ['French 11', 'Unassigned Teacher']
Course 29 Section 3 ['French 9', 'Unassigned Teacher']
Course 36 Section 4 ['Literary Studies 11', 'Unassigned Teacher']
Course 48 Section 2 ['Visual Arts 8', 'Harms']

Block 2 courses are
Course 1 Section 1 ['AP Biology', 'Butler']
Course 8 Section 2 ['Art Studio 2', 'Harms']
Course 18 Section 3 ['Drama 9', 'Unassigned Teacher']
Course 21 Section 4 ['English 8', 'Unassigned Teacher']
Course 23 Section 3 ['English Studies 12', 'Unassigned Teacher']
Course 29 Section 1 ['French 9', 'Unassigned Teacher']
Course 31 Section 3 ['Guided Study Block', 'Unassigned Teacher']
Course 33 Section 1 ['Digital Arts 9', 'Holowka']
Course 36 Section 3 ['Li